Import Dependencies

In [152]:
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import requests
from bs4 import BeautifulSoup
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

Scrpa meta description by URL

In [153]:


def scrape_website(url):


    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for non-200 status codes

        soup = BeautifulSoup(response.content, 'html.parser')

        data = {
            'url': url,

            'meta_description': None,

        }


        meta_description = soup.find('meta', attrs={'name': 'description'})
        if meta_description:
            data['meta_description'] = meta_description['content']
            text = data["meta_description"]

        return text

    except requests.exceptions.RequestException as e:
        print(f"Error scraping website: {e}")
        return text

# Example usage
# url = 'https://www.allheartweb.com'
url = input('Input url : ')
text = scrape_website(url)

if text:
    print(text)
else:
    print("Scraping failed.")


Input url : https://www.allheartweb.com
Redefining Domain Intelligence with AllHeartWeb.com


Import training data

In [154]:
data=pd.read_csv("website_classification.csv")

In [155]:
data.head()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1408 entries, 0 to 1407
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Unnamed: 0            1408 non-null   int64 
 1   website_url           1408 non-null   object
 2   cleaned_website_text  1408 non-null   object
 3   Category              1408 non-null   object
dtypes: int64(1), object(3)
memory usage: 44.1+ KB


remove unnecessary columns from Code

In [156]:
data=data.drop(["Unnamed: 0","website_url"],axis=1)

In [157]:
data.cleaned_website_text.head()

0    official site good hotel accommodation big sav...
1    expedia hotel book sites like use vacation wor...
2    tripadvisor hotel book sites like previously d...
3    cheap flights search compare flights momondo f...
4    bot create free account create free account si...
Name: cleaned_website_text, dtype: object

Remove duplicates

In [158]:
data=data.drop_duplicates()

Seggregate columns

In [159]:
feature=data['cleaned_website_text']
target=data['Category']
feature=pd.DataFrame(feature)

Encoded target column

In [160]:
label_encoder=LabelEncoder()
target=label_encoder.fit_transform(target)

Data cleaning

In [161]:
## Cleaning the feature column
def cleaning_processing(sentance):
    stop_words=set(stopwords.words('english'))
    sentance=word_tokenize(sentance)  # Split the sentence in two words
    sen=[]
    for i in sentance:
        sentance=i.lower()   # convert into lower_case
        sentance=re.sub(r"[^a-zA-Z]","",sentance) # Remove special_charater
        if sentance not in stop_words:
            if sentance != '': # Remove empty string
                sen.append(sentance)
    filtered_sentence = " ".join(sen)  # Converting a list of words into a string
    return filtered_sentence

In [162]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [163]:
feature["cleaned_website_text"]=feature["cleaned_website_text"].apply(cleaning_processing)

In [164]:
# text = text["meta_description"]
# text = "tripadvisor hotel book sites like previously deal predominantly restaurant bar review recent year tripadvisor begin offer hotel reservation fine result unlike tripadvisor alternative sites tripadvisor blog travelsites hotel book sites tripadvisor home visit site vote previously deal predominantly restaurant bar review recent year tripadvisor begin offer hotel reservation fine result unlike tripadvisor compare cost number book website include give option view deal order find good price like tripadvisor find stay accommodation detail bit thin website review system useful choose spend hard earn cash select review depend language traveller type time year give good idea like stay hotel precisely plan tripadvisor give option find key word review example good location type location traveller say comment website include contact detail hotel link hotel specific website give user feeling restrict book tripadvisor want photo place divide professional photo traveller photo add transparency make booking know happy tripadvisor feature tell people look property aggressive pop up partner company believe make user feel ease make reservation website time feel convoluted review post trust tripadvisor great tool find book accommodation david jones professional travel writer travel enthusiast travel world twice share firsthand knowledge relate travel spending time abroad pro allow user compare result website help find good deal user review refine filter order find specifically look user professional photo user photo good sense hotel like contact info hotel website include con little information type accommodation compare book website website feel convoluted confusing time ranks hotel user view user review trust expedia agoda priceline orbitz travelocity hotwire hotelscombined momondo hotels onetravel hotels hotel booking sites click travel list travel sites check big date travel list good travel site click good travel sites blog world good travel si"

Preprocessing Scrapped Text

In [165]:
text = cleaning_processing(text)
list1 = [text]
con_vector=TfidfVectorizer(max_features = 140)
# text = list1
text=con_vector.fit_transform(list1)


In [166]:
maxfeatures = text.shape[1]

maxfeatures is the features extracted from the scrapped text
we require the no. of training features to be the same as the no. of input features

In [167]:

con_vector=TfidfVectorizer(max_features = maxfeatures)
feature=con_vector.fit_transform(feature["cleaned_website_text"])

Test train split

In [168]:
x_train,x_test,y_train,y_test=train_test_split(feature,target,test_size=0.2, random_state=42)

In [169]:
print("X_train shape:", x_train.shape)
print("X_test shape:", x_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (1100, 4)
X_test shape: (275, 4)
y_train shape: (1100,)
y_test shape: (275,)


Training the Model , KNN in this case

In [170]:
classifier1=KNeighborsClassifier()
# x_train = x_train[0:30]
classifier1.fit(x_train,y_train)
predict1=classifier1.predict(x_test)
accuracy1 = accuracy_score(y_test, predict1)
print("KNeighborsClassifier accuracy score: ",accuracy1)

KNeighborsClassifier accuracy score:  0.11272727272727273


defining categories

In [171]:
category_labels = data['Category'].values  # Get the category column as a NumPy array
category_labels_1d = set(category_labels)
category_labels_1d = category_labels.flatten()

# category_labels_1d

processing input

In [172]:

model = classifier1

new_website_text = text
print(new_website_text.shape)
# Predict the category using the loaded model
predicted_category = model.predict(new_website_text)[0]


label_encoder = LabelEncoder()
label_encoder.fit(category_labels_1d)

category_names = label_encoder.inverse_transform([predicted_category])

print("Predicted category:", category_names[0])


(1, 4)
Predicted category: Games
